In [ ]:
import os
from numpy import sqrt
from pyroot_module import *


### Setting PATH and loading libraries for Delphes and ExRootAnalysis 

In [ ]:
from datetime import date
#path of folder to save the images
imgPath = './images/spin0/%s/'%(date.today().strftime('%Y-%m-%d'))

### Reading trees and selecting branches

In [ ]:
# getting signal and background trees
# signal
backgroundPath = './data/background/'
signalPath = '/home/santiago/VBF_DMSimp_spin0_EWKExcluded/Events/run_18/'

fs = [] #Signal files
imgFold = [] #Images forlders
for f in os.listdir(signalPath):
    if os.path.isfile(signalPath + f) and '.root' in f:
        fs.append(ROOT.TFile(signalPath + f))
    
        #create folder to save images
        imgFold.append((imgPath+f).replace('.root',''))
        if not os.path.exists(imgFold[-1]):
            os.makedirs(imgFold[-1])
    
        
#Signal trees
s = [f.Get("Delphes") for f in fs]

# background
b = ROOT.TChain("Delphes")
b.Add(backgroundPath + "santiago_run_02.root") #no
b.Add(backgroundPath + "santiago_run_05.root") #no
b.Add(backgroundPath + "santiago_run_07.root") #no

# number of signal and background events 
nb = int(b.GetEntries())
ns = [int(k.GetEntries()) for k in s]

# number of bins to book histograms
nbBins = int(sqrt(nb))+1
nsBins = [int(sqrt(n))+1 for n in ns] 

# reading trees and selecting branches
treeB = ROOT.ExRootTreeReader(b)
treeS = [ROOT.ExRootTreeReader(t) for t in s]

jetB = treeB.UseBranch("Jet")
jetS = [tree.UseBranch("Jet") for tree in treeS]

metB = treeB.UseBranch("MissingET")
metS = [tree.UseBranch("MissingET") for tree in treeS]

HtS = [tree.UseBranch("ScalarHT") for tree in treeS]
HtB = treeB.UseBranch("ScalarHT")

# number of jets to use
numJets = 5

### Book histograms

In [ ]:
#---------------------------
#           signal
#---------------------------
hsDeltaEtaJet = [ROOT.TH1F("hsDeltaEtaJet_{0}".format(j), "Delta_Eta_Jet12_{0}".format(j), 50,0,10)
                for j in range(len(s))]

#Delta Eta for the two jets with highest invariant mass
hsDeltaEtaMaxIM = [ROOT.TH1F("hsDeltaEtaM_{0}".format(j), "Delta_Eta_M_{0}".format(j), 50,0,10) 
                   for j in range(len(s))]
hsMass = [ROOT.TH1F("hSMass_{0}".format(j), "M(j1,j2)_{0}".format(j), 65,0,4000)
         for j in range(len(s))]

# Highest invariant mass between to jets
hsMassMax = [ROOT.TH1F("hsMjj_{0}".format(j), "Max_M_{0}".format(j), 65,0,4000)
            for j in range(len(s))]

hsDeltaPhi = [ROOT.TH1F("hsDeltaPhi_{0}".format(j), "delta_Phi_{0}".format(j), 50, -1, 4)
              for j in range(len(s))]

# Delta phi for the two jets with highest invariant mass
hsDeltaPhiMax = [ROOT.TH1F("hsDeltaPhiMax_{0}".format(j), "Max_delta_phi_{0}".format(j), 50, -1, 4)
                 for j in range(len(s))]

hsDeltaPhiMetJet = [[ROOT.TH1F("hsDelPhiMet_j{0}_{1}".format(i,j),"dMetJ{0}_{1}".format(i,j), 50, -1, 4) 
                     for i in range(numJets)] 
                    for j in range(len(s))]

hsMetET = [ROOT.TH1F("hsMetET_{0}".format(j), "MET_ET_{0}".format(j), 50, 0.0, 2000)
           for j in range(len(s))]

hsHt = [ROOT.TH1F("hsHT_{0}".format(j),"HT_{0}".format(j),50,0,2000) for j in range(len(s))]

#---------------------------
#        background
#---------------------------
hbDeltaEtaJet = ROOT.TH1F("hbDeltaEtaJet", "Delta_Eta_Jet12", 50,0,10)
hbDeltaEtaMaxIM = ROOT.TH1F("hbDeltaEtaMaxIM", "Delta_Eta_MaxIM", 50,0,10)
hbMass = ROOT.TH1F("hbMass", "M(j1,j2)", 65,0,4000)
hbMassMax = ROOT.TH1F("hbMjj", "Max_M", 65,0,4000)
hbDeltaPhi = ROOT.TH1F("hbDeltaPhi", "delta_Phi", 50, -1, 4)
hbDeltaPhiMax = ROOT.TH1F("hbDeltaPhiMax", "Max_delta_phi", 50, -1, 4)
hbDeltaPhiMetJet = [ROOT.TH1F("hbDelPhiMet_j{0}".format(i),"dMetJ{0}".format(i), 50, -1, 4) 
                    for i in range(numJets)]
hbMetET = ROOT.TH1F("hbMetET", "MET_ET", 50, 0.0, 2000)
hbHt = ROOT.TH1F("hbHT","HT",50,0,2000)

### Creating dictionaries for the cuts

In [ ]:
s_c = { 'cut0':0, 'cut1':0, 'cut2':0, 'cut3':0, 'cut4':0, 'cut5':0, 'cut6':0, 'cut7':0}
b_c = { 'cut0':0, 'cut1':0, 'cut2':0, 'cut3':0, 'cut4':0, 'cut5':0, 'cut6':0, 'cut7':0}
c_c = { 'cut0': r'$no\ cuts$',
        'cut1': r'$H_T$',        
        'cut2': r'$N^o Jets \geq 2$',
        'cut3': r'$P_T(J_i)>30\ i=0,1$',
        'cut4': r'$\eta (J_0)*\eta (J_1) < 0$',
        'cut5': r'$|\Delta \phi (J_0,J_1)| \geq 2.3$',
        'cut6': r'$max(m(J_i,J_j)) \geq 1000$',
        'cut7': r'$min(|\Delta\phi(MET,J_i)|),\ i=0...4$'}

### Selecting events and filling histograms

In [ ]:
# Loop over all signal events
for k in range(len(ns)):
    print(k)
    for event in range(ns[k]):
        #Load selected branches with data from specified event
        treeS[k].ReadEntry(event)

        # cuts
        s_c['cut0'] += 1
        
        if HtS[k].At(0).HT <= 200 : continue
        s_c['cut1'] += 1
        
        # select events with at least 2 jets
        if jetS[k].GetEntries() < 2: continue
        s_c['cut2'] += 1
        
        if not (jetS[k].At(0).PT > 30 and jetS[k].At(1).PT > 30 and
                jetS[k].At(0).Eta < 5 and jetS[k].At(1).Eta < 5): continue
        s_c['cut3'] += 1
        
        # select events with jets in opposite hemispheres 
        if (jetS[k].At(0).Eta * jetS[k].At(1).Eta) >= 0: continue
        s_c['cut4'] += 1
        
        if abs(DeltaPhi(jetS[k].At(0).Phi,jetS[k].At(1).Phi)) < 2.3: continue
        s_c['cut5'] += 1
            
        i = 0
        j = 0
        JsMass = 0
        NJets = jetS[k].GetEntries() if jetS[k].GetEntries() <= numJets else numJets
        dPhiMetJet = []
        for n in range(NJets):
            # selecting the two jet system 
            # with the largest invariant mass
            dPhiMetJet.append(abs(DeltaPhi(metS[k].At(0).Phi,jetS[k].At(n).Phi)))            
            for m in range(NJets):
                if n==m:continue
                a = InvariantMass(jetS[k],n,m) 
                if a > JsMass:
                    i,j = m,n
                    JsMass = a
                              
        # Cut for maximum invariant mass
        if JsMass < 1000 : continue
        s_c['cut6'] += 1
        
        if min(dPhiMetJet) <= 0.5 : continue
        s_c['cut7'] += 1
                              
        for n in range(NJets):
            hsDeltaPhiMetJet[k][n].Fill(dPhiMetJet[n])                              
        hsDeltaEtaJet[k].Fill(abs(jetS[k].At(0).Eta-jetS[k].At(1).Eta))
        hsDeltaEtaMaxIM[k].Fill(abs(jetS[k].At(i).Eta-jetS[k].At(j).Eta))
        hsMass[k].Fill(InvariantMass(jetS[k],0,1))
        hsMassMax[k].Fill(JsMass)
        hsDeltaPhi[k].Fill(abs(DeltaPhi(jetS[k].At(0).Phi,jetS[k].At(1).Phi)))
        hsDeltaPhiMax[k].Fill(abs(DeltaPhi(jetS[k].At(i).Phi,jetS[k].At(j).Phi)))
        hsMetET[k].Fill(metS[k].At(0).MET)
        hsHt[k].Fill(HtS[k].At(0).HT)

In [ ]:
# Loop over all background events 
for event in range(nb):
    # load selected branches with data from specified event
    treeB.ReadEntry(event)
    
    # cuts
    b_c['cut0'] += 1
    
    if HtB.At(0).HT <= 200: continue
    b_c['cut1'] += 1
    
    # select events with at least 2 jets
    if jetB.GetEntries() < 2: continue
    b_c['cut2'] += 1
    
    if not (jetB.At(0).PT > 30 and jetB.At(1).PT > 30 and
            jetB.At(0).Eta < 5 and jetB.At(1).Eta < 5): continue
    b_c['cut3'] += 1
    
    # select events with jets in opposite hemispheres 
    if (jetB.At(0).Eta * jetB.At(1).Eta) >= 0: continue
    b_c['cut4'] += 1
    
    if abs(DeltaPhi(jetB.At(0).Phi,jetB.At(1).Phi)) < 2.3: continue
    b_c['cut5'] += 1
        
    i = 0
    j = 0
    JbMass = 0
    NJets = jetB.GetEntries() if jetB.GetEntries() <= numJets else numJets
    dPhiMetJet = []
    for n in range(NJets):
        # selecting the two jet system 
        # with the largest invariant mass
        dPhiMetJet.append(abs(DeltaPhi(metB.At(0).Phi,jetB.At(n).Phi)))
        for m in range(NJets):
            if n==m:continue
            a = InvariantMass(jetB,n,m) 
            if a > JbMass:
                i,j = m,n
                JbMass = a

   # Cut for maximum invariant mass
    if JbMass < 1000 : continue
    b_c['cut6'] += 1
    
    if min(dPhiMetJet) <= 0.5 : continue
    b_c['cut7'] += 1
    
    for n in range(NJets):         
        hbDeltaPhiMetJet[n].Fill(dPhiMetJet[n])        
    hbDeltaEtaJet.Fill(abs(jetB.At(0).Eta-jetB.At(1).Eta))
    hbDeltaEtaMaxIM.Fill((abs(jetB.At(i).Eta-jetB.At(j).Eta)))
    hbMass.Fill(InvariantMass(jetB,0,1))
    hbMassMax.Fill(JbMass)
    hbDeltaPhi.Fill(abs(DeltaPhi(jetB.At(0).Phi,jetB.At(1).Phi)))
    hbDeltaPhiMax.Fill(abs(DeltaPhi(jetB.At(i).Phi,jetB.At(j).Phi)))
    hbMetET.Fill(metB.At(0).MET)
    hbHt.Fill(HtB.At(0).HT)

## Cut flow

In [ ]:
f1,f2 = CutFlow(s_c,b_c,c_c,8,ns[0],nb)
print('ns = {}'.format(ns[0]))
print('nb = {}'.format(nb))

In [ ]:
f1

In [ ]:
f2

## Delta_Eta

In [ ]:
fList = open(imgPath + "fileList.txt","w+")
k=0
for f in os.listdir(signalPath):
    if os.path.isfile(signalPath + f) and '.root' in f:
        fList.write('%d\t%s\r\n'%(k,f))
        k+=1
fList.close()

for k in range(len(hsDeltaEtaJet)):
    HzsDEta = hsDeltaEtaJet[k].Clone()
    HzbDEta = hbDeltaEtaJet.Clone()
#     Histos(HzsDEta,HzbDEta, imgFold[k] + '/deltaEta_j12.png')
    PlotHistos(HzsDEta,HzbDEta,save=True,savePath=imgFold[k])
#     Hz = Sig(HzsDEta, HzbDEta, ns=ns[k], nb=nb)
#     Histo(Hz,imgFold[k] + '/z_DeltaEta_j1_j2.png')
    Hz = Significance(HzsDEta, HzbDEta, ns=ns[k], nb=nb, lessThan=True)
    PlotHisto(Hz,save=True,savePath=imgFold[k])

### Delta Eta maximum invariant mass jet

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    HzsMass = hsDeltaEtaMaxIM[k].Clone()
    HzbMass = hbDeltaEtaMaxIM.Clone()
#     Histos(HzsMass,HzbMass, imgFold[k] + '/deltaEta_MaxIM.png')
#     Hz = Sig(HzsMass, HzbMass, ns=ns[k], nb=nb)
#     Histo(Hz,imgFold[k] + '/z_DeltaEta_maxMassInv.png')
    PlotHistos(HzsMass,HzbMass,save=True,savePath=imgFold[k])
    Hz = Significance(HzsMass, HzbMass, ns=ns[k], nb=nb, lessThan=True)
    PlotHisto(Hz,save=True,savePath=imgFold[k])

## Invariant Mass

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    HsMass = hsMass[k].Clone()
    HbMass = hbMass.Clone()
#     Histos(HsMass,HbMass, imgFold[k] + '/m_j1_j2.png')
#     HzM = Sig(HsMass, HbMass, ns=ns[k], nb=nb)
#     Histo(HzM,imgFold[k] + '/z_m_j1_j2.png')
    PlotHistos(HsMass,HbMass, save=True,savePath=imgFold[k])
    HzM = Significance(HsMass, HbMass, ns=ns[k], nb=nb)
    PlotHisto(HzM,save=True,savePath=imgFold[k])

### Maximum invariant mass

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    HsMass = hsMassMax[k].Clone()
    HbMass = hbMassMax.Clone()
#     Histos(HsMass,HbMass, imgFold[k] + '/m_max.png')
#     HzM = Sig(HsMass, HbMass, ns=ns[k], nb=nb)
#     Histo(HzM,imgFold[k] + '/z_m_maxMassInv.png')
    PlotHistos(HsMass,HbMass,save=True,savePath=imgFold[k])
    HzM = Significance(HsMass, HbMass, ns=ns[k], nb=nb)
    PlotHisto(HzM,save=True,savePath=imgFold[k])

### Delta Phi

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    HzsPhi = hsDeltaPhi[k].Clone()
    HzbPhi = hbDeltaPhi.Clone()
#     Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi(j1,j2).png')
    PlotHistos(HzsPhi,HzbPhi, save=True,savePath=imgFold[k])
    Hz = Significance(HzsPhi, HzbPhi, ns=ns[k], nb=nb)
    PlotHisto(Hz,save=True,savePath=imgFold[k])

### Delta Phi maximum invariant mass for two jets

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    HzsPhi = hsDeltaPhiMax[k].Clone()
    HzbPhi = hbDeltaPhiMax.Clone()
#     Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi__max.png')
    PlotHistos(HzsPhi,HzbPhi, save=True,savePath=imgFold[k])
    
    Hz = Significance(HzsPhi, HzbPhi, ns=ns[k], nb=nb)
    PlotHisto(Hz,save=True,savePath=imgFold[k])

### DeltaPhi(Met,Jet)

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    for n in range(numJets):
        HzsPhi = hsDeltaPhiMetJet[k][n].Clone()
        HzbPhi = hbDeltaPhiMetJet[n].Clone()
        PlotHistos(HzsPhi,HzbPhi, save=True,savePath=imgFold[k])
    
        Hz = Significance(HzsPhi, HzbPhi, ns=ns[k], nb=nb,lessThan=True)
        PlotHisto(Hz,save=True,savePath=imgFold[k])

### MET


In [ ]:
for k in range(len(hsDeltaEtaJet)):
    Hzs = hsMetET[k].Clone()
    Hzb = hbMetET.Clone()
#     Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi__max.png')
    PlotHistos(Hzs,Hzb, save=True,savePath=imgFold[k])
    Hz = Significance(Hzs, Hzb, ns=ns[k], nb=nb)
    PlotHisto(Hz,save=True,savePath=imgFold[k])

### HT

In [ ]:
for k in range(len(hsDeltaEtaJet)):
    Hzs = hsHt[k].Clone()
    Hzb = hbHt.Clone()
#     Histos(HzsPhi,HzbPhi, imgFold[k] + '/deltaPhi__max.png')
    PlotHistos(Hzs,Hzb, save=True,savePath=imgFold[k])
    Hz = Significance(Hzs, Hzb, ns=ns[k], nb=nb)
    PlotHisto(Hz,save=True,savePath=imgFold[k])